In [4]:
import base64
import datetime
import io

from jupyter_dash import JupyterDash
from dash import no_update
from dash.dependencies import Input, Output, State
from dash import dcc
from dash import html
from dash import dash_table
from dash.dash_table.Format import Group
import plotly.express as px
import dash_table_experiments as dt


import pandas_bokeh
import matplotlib.pyplot as plt
import pgeocode
import geopandas as gpd
from shapely.geometry import Point
from geopandas import GeoDataFrame
pandas_bokeh.output_notebook()
import plotly.graph_objects as go
import plotly.express as px

import pandas as pd

import validation_functions as vf


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets,
                suppress_callback_exceptions=True)

app.layout = html.Div([ # this code section taken from Dash docs https://dash.plotly.com/dash-core-components/upload
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Drag and Drop or ',
            html.A('Select Files')
        ]),
        style={
            'width': '100%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-div'),
    html.Div(id='output-datatable'),
])


def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
            
            data_validation = vf.data_prep(df)
        
            data_validation_sample = vf.mf_sample(data_validation)

            data_final = vf.dict_final(data_validation_sample)

            data_validation_sample = vf.drop_zip_id(data_validation_sample)

            df = vf.get_users(data_validation_sample, data_final)
            
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),
        html.Button(id="submit-button", children="Create Graph"),
        
        dash_table.DataTable(
            data=df.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in df.columns],
            page_size=15
        ),
        dcc.Store(id='stored-data', data=df.to_dict('records')),

        html.Hr(),  # horizontal line
    ])


@app.callback(Output('output-datatable', 'children'),
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'),
              State('upload-data', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children


@app.callback(Output('output-div', 'children'),
              Input('submit-button','n_clicks'),
              State('stored-data','data'))

def make_graphs(n, data):
    if n is None:
        return no_update
    else:
        df = pd.DataFrame.from_records(data)
        zipcodes = df.groupby(['ZIPCode']).size().reset_index(name='counts')
        zipcodes['ZIPCode'] = zipcodes['ZIPCode'].astype(str)

        nomi = pgeocode.Nominatim('us')

        zipcodes['Latitude'] = (nomi.query_postal_code(zipcodes['ZIPCode'].tolist()).latitude)
        zipcodes['Longitude'] = (nomi.query_postal_code(zipcodes['ZIPCode'].tolist()).longitude)

        fig = go.Figure(data=go.Scattergeo(
                lon = zipcodes['Longitude'],
                lat = zipcodes['Latitude'],
                text = zipcodes['ZIPCode'],
                mode = 'markers',
                marker_color = zipcodes['counts'],
                opacity = 0.8
                ))

        fig.update_layout(
                title = 'ZIPCode Distribution',
                geo_scope='usa',
            )
        fig.show()
        return dcc.Graph(figure=fig)

app.run_server(mode='jupyterlab', port=8040)

Loading BokehJS ...

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
